In [1]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.interpolate import interp1d
import scipy.stats as st
from scipy.stats import gaussian_kde
from scipy.signal import find_peaks_cwt

In [2]:
from pathlib import Path

import pandas as pd

DATA_ROOT = Path('..') / 'data'

dfs = []
activity_labels = ['bed', 'chair', 'lying', 'ambulating']
default_names = ['time', 'front', 'vertical', 'lateral', 'sensor_id', 'rssi', 'phase', 'frequency', 'activity']
for data_file in Path(DATA_ROOT).rglob('d[12]p??[FM]'):
    df = pd.read_csv(data_file, names=default_names)
    df['activity_label'] = df['activity'].apply(lambda i: activity_labels[i - 1])
    df['gender_label'] = str(data_file)[-1]
    df['participant'] = data_file.name
    
    # Add a column indicating order of the activities for a particiapnt.
    df = df.sort_values(by=['time'])
    df['activity_sequence'] = (df['activity'].shift(1) != df['activity']).cumsum()
    dfs.append(df)

sensor_df = pd.concat(dfs, axis='index')
sensor_df = sensor_df.sort_values(by=['participant', 'time'])

sensor_df.head()

,time,front,vertical,lateral,sensor_id,rssi,phase,frequency,activity,activity_label,gender_label,participant,activity_sequence
0,0.0,0.27203,1.00820,-0.082102,1,-63.5,2.4252,924.25,1,bed,M,d1p01M,1
0,0.0,0.27203,1.00820,-0.082102,1,-63.5,2.4252,924.25,1,bed,M,d1p01M,1
1,0.5,0.27203,1.00820,-0.082102,1,-63.0,4.7369,921.75,1,bed,M,d1p01M,1
1,0.5,0.27203,1.00820,-0.082102,1,-63.0,4.7369,921.75,1,bed,M,d1p01M,1
2,1.5,0.44791,0.91636,-0.013684,1,-63.5,3.0311,923.75,1,bed,M,d1p01M,1


### Plotting interpolated data on a time normalized scale

Due to the activities having different durations with different numbers of data points we can consder normalizing the duration of each activity on to a unit scale with a fixed number of points, and interpolating the data on this scale.

First we define some useful functions:

In [3]:
def interpolate(y, xnew):
    """Linear interpolation."""
    
    # We need at least two samples
    if len(y) == 1:
        y = [y[0]] * 2
        x = [min(xnew), max(xnew)]
    else:
        x = np.linspace(min(xnew), max(xnew), len(y))

    # f is a function which approximates the relationship between x and y
    f = interp1d(x, y)

    # Get the new y values.
    newy = f(xnew)
    return newy

def interpolate_sensor_data(df, n):
    """Iterpolate the sensor data using n points on a normalized time scale."""
    data = {}
    for column in ['participant', 'activity', 'activity_label', 'activity_sequence']:
        data[column] = [df.iloc[0][column]] * n
    t_norm = np.linspace(0, 1, n)
    data['t_norm'] = t_norm
    for column in ['front', 'lateral', 'vertical']:
        data[column] = interpolate(df[column].values, t_norm)
    return pd.DataFrame(data)

def jitter(size, mean=0., sd=1.):
    """Create some 'jitter' values to perturb data points to increase visibility."""
    return np.random.normal(mean, sd, size=size)

Then we can create a 'time-normalized' data frame with the sensor data interpolated along equal spaced points on the unit interval:

In [4]:
norm_df = (sensor_df.groupby(['participant', 'activity_sequence'])
           .apply(lambda df: interpolate_sensor_data(df, 40))
           .reset_index(drop=True))
norm_df.head()

,participant,activity,activity_label,activity_sequence,t_norm,front,lateral,vertical
0,d1p01M,1,bed,1,0.000000,0.272030,-0.082102,1.008200
1,d1p01M,1,bed,1,0.025641,0.272030,-0.082102,1.008200
2,d1p01M,1,bed,1,0.051282,0.447910,-0.013684,0.916360
3,d1p01M,1,bed,1,0.076923,0.342380,-0.059296,0.962290
4,d1p01M,1,bed,1,0.102564,0.336067,-0.061343,0.968473


We can then plot the data. There are still a lot of data points, so the interactive plot may be slight slow to respond depending on the browser:

In [5]:
colors = px.colors.qualitative.D3
values = ['front', 'lateral', 'vertical']
SENSOR_COLORS = {
    value: colors[i] for i, value in enumerate(values)}

activity_labels = sorted(norm_df['activity_label'].unique())

fig = make_subplots(
    rows=2, 
    cols=2, 
    vertical_spacing=0.1,
    horizontal_spacing=0.1,
    subplot_titles=[label.capitalize() for label in activity_labels])

first = True
mode = 'lines+markers'
for _, activity_df in norm_df.groupby(['participant', 'activity_sequence']):
    activity_label = activity_df['activity_label'].values[0]
    i = activity_labels.index(activity_label)
    
    row = i // 2 + 1
    col = i % 2 + 1
    for sensor in ['front', 'lateral', 'vertical']:
        fig.add_trace(
            go.Scattergl(
                x=activity_df['t_norm'] + jitter(len(activity_df), sd=0.01), 
                y=activity_df[sensor],
                marker_color=SENSOR_COLORS[sensor],
                opacity=0.0625,
                name=sensor,
                showlegend=False,
                legendgroup=sensor,
                mode=mode), row=row, col=col)
    first = False

# Just for legend :p
for sensor in ['front', 'lateral', 'vertical']:
    fig.add_trace(
        go.Scattergl(
            x=[None], 
            y=[None],
            marker_color=SENSOR_COLORS[sensor],
            name=sensor.capitalize(),
            showlegend=True,
            legendgroup=sensor,
            mode=mode))
    
fig.update_yaxes(range=[-2,2], showgrid=False, zeroline=True, showticklabels=False)
fig.update_xaxes(showgrid=False, showticklabels=False, zeroline=True)
fig.update_layout(
    width=800, 
    height=600, 
    margin=dict(l=16, r=16, t=80, b=16),
    legend_title='<b>Sensor</b>',
    title='<b>Interpolated and time-normalized sensor data for all participants</b>')
fig.show(renderer='iframe_connected')


### Idealised plots

Assuming the sensor data is generated by a univariate Gaussian distribution (it isn't - the density plots show this), we can estimate the mean and 95% confidence intervals for the sensor data at each time point (we should use a sliding window, but the current approach is the same as a non-overlapping sliding window). This enables an 'idealised' plot to be created. The resulting plot should be regarded with caution due to the incorrect assumption of normality, but may still be instructive.

In [6]:
t_norms = sorted(norm_df['t_norm'].unique())
rows = []
for activity_label in norm_df['activity_label'].unique():
    activity_df = norm_df[norm_df['activity_label'] == activity_label]
    for t in t_norms:
        row = {'time': t, 'activity_label': activity_label}
        for sensor in ['front', 'lateral', 'vertical']:
            sample = activity_df[activity_df['t_norm'] == t][sensor]
            mid = np.mean(sample)
            high, low = st.t.interval(0.95, len(sample) - 1, loc=mid, scale=np.std(sample))
            row[f'{sensor}-mid'] = mid
            row[f'{sensor}-low'] = low
            row[f'{sensor}-high'] = high
        rows.append(row)
sample_df = pd.DataFrame(rows)
sample_df.head()

,time,activity_label,front-mid,front-low,front-high,lateral-mid,lateral-low,lateral-high,vertical-mid,vertical-low,vertical-high
0,0.000000,bed,0.452491,1.149618,-0.244637,-0.105740,0.611552,-0.823032,0.772841,1.228377,0.317305
1,0.025641,bed,0.431115,1.118776,-0.256547,-0.104309,0.587642,-0.796261,0.791285,1.231267,0.351303
2,0.051282,bed,0.426257,1.077772,-0.225257,-0.106031,0.575448,-0.787511,0.803043,1.230670,0.375416
3,0.076923,bed,0.433066,1.074376,-0.208245,-0.099114,0.578245,-0.776473,0.805252,1.224627,0.385878
4,0.102564,bed,0.432412,1.072652,-0.207828,-0.099644,0.575789,-0.775076,0.808181,1.221945,0.394417


In [7]:
activity_labels = sorted(sample_df['activity_label'].unique())

subplot_titles = np.reshape(activity_labels, (2, 2))
    
fig = make_subplots(
    rows=2, 
    cols=2, 
    subplot_titles=[title.capitalize() for title in subplot_titles.flatten('C')],
    vertical_spacing=0.1,
    horizontal_spacing=0.1)

for row, col in [(1, 1), (1, 2), (2, 1), (2, 2)]:
    activity_label = subplot_titles[row - 1, col - 1]
    activity_df = sample_df[sample_df['activity_label'] == activity_label]
    for sensor in ['front', 'lateral', 'vertical']:
        fig.add_trace(
            go.Scatter(
                x=pd.concat((activity_df['time'], activity_df['time'][::-1])), 
                y=pd.concat((activity_df[f'{sensor}-low'], activity_df[f'{sensor}-high'][::-1])),
                fillcolor=SENSOR_COLORS[sensor],
                opacity=0.25,
                line_width=0,
                mode='lines',
                name=sensor,
                fill='toself',
                showlegend=False,
                legendgroup=sensor),
            row=row,
            col=col)
        
        fig.add_trace(
            go.Scatter(
                x=activity_df['time'], 
                y=activity_df[f'{sensor}-mid'],
                line_color=SENSOR_COLORS[sensor],
                mode='lines',
                name=sensor.capitalize(),
                showlegend=row == 1 and col == 1,
                legendgroup=sensor),
            row=row,
            col=col)
fig.update_yaxes(range=[-2,2], showgrid=False, zeroline=True, showticklabels=False)
fig.update_xaxes(range=[-0.05, 1.05], showgrid=False, showticklabels=False, zeroline=True)
fig.update_layout(
    width=800, 
    height=600, 
    margin=dict(l=16, r=16, t=80, b=16),
    title='<b>Interpolated and time-normalized sensor data with 95% CIs',
    legend_title='<b>Sensor</b>')
fig.show(renderer='iframe_connected')